# Place your sample.json file from client nearby first

In [2]:
samples_path = 'raw/samples_1697144368492.json'
ds_path = 'ds'

## Export to yolo format

In [4]:
import json
import base64
from PIL import Image
from io import BytesIO
import os

with open(samples_path) as samples_file:
    samples = json.load(samples_file)

counter = {}
for sample in samples:
    # Remove 'data:image/png;base64,'
    imagestr = sample['image'][sample['image'].find(",")+1:]
    # Convert base64 image to PIL 
    imgdata = base64.b64decode(imagestr)
    img = Image.open(BytesIO(imgdata)).convert('RGB')
    
    gesture = sample["gesture"]
    if gesture in counter: 
        counter[gesture] += 1
    else :
        counter[gesture] = 0
    
    # Crop the hand from image 
    box = sample['box']
    img = img.crop((box['x']-box['x']/10, box['y']-box['y']/10, box['x'] + box['w']+box['x']/10, box['y'] + box['h']+box['y']/10))
    img = img.resize((416, 416))
    
    # Save each class to separate folder inside ds_path
    save_path = f'{ds_path}/{sample["gesture"]}'
    if not os.path.exists(save_path):
            os.makedirs(save_path)
    img.save(save_path + f'/{sample["gesture"]}_{counter.get(gesture)}.jpg')

## Augment data

In [10]:
import Augmentor

p = Augmentor.Pipeline(ds_path, output_directory='../augmented')
p.rotate(probability=0.7, max_left_rotation=15, max_right_rotation=15)
p.zoom(probability=0.5, min_factor=1.05, max_factor=1.2)
p.flip_left_right(probability=0.5)

# Make 10000 in total from input and save this to 'augmented' dir
p.sample(1000)

Initialised with 87 image(s) found.
Output directory set to ds/../augmented.

Processing <PIL.Image.Image image mode=RGB size=416x416 at 0x7F207C5A7310>: 100%|██████████| 1000/1000 [00:02<00:00, 360.29 Samples/s]                  


## Split to train and val

In [11]:
import splitfolders

splitfolders.ratio("augmented", output=f"{ds_path}/ds", ratio=(.8, .2), group_prefix=None, move=False)

Copying files: 1000 files [00:00, 11319.77 files/s]
